## Pregunta de Negocio - Analistes de Finances i Risc Crediticio

> ¿Qué **umbrales de saldo** podrían indicar mayor **riesgo de morosidad**?


### Carga de datos

In [2]:
"""Librerías que pueden ser necesarias"""

# Manejo de datos
import pandas as pd                 # Análisis y manipulación de datos en tablas
import numpy as np                  # Cálculos numéricos y matrices
import os                           # Gestión de rutas de archivos

# Visualización de datos
import matplotlib.pyplot as plt     # Gráficos básicos en 2D
import seaborn as sns               # Gráficos estadísticos mejorados
import plotly.express as px         # Gráficos interactivos simplificados
import plotly.graph_objects as go   # Control avanzado de gráficos en Plotly
import plotly.io as pio             # Interfaz de entrada/salida de Plotly
import squarify                     # Visualización de diagramas de "treemap"

# Procesado y análisis
from scipy import stats

#Paleta de colores oficial
custom_palette = ["#2E2F36", "#5F6F81", "#AABBC8", "#DCE3EA", "#2CCED1"]

In [3]:
df = pd.read_csv("../Data/df_250519.csv")

# KPIs

- saldo medio gral
- saldo medio por decil
- % clientes por tasa de riesgo
- % clientes con hipoteca
- % clientes con préstamo
- % clientes con hipoteca + préstamo
- tasa contratación depósito
- tasa incumplimiento
- probabilidad de incumplimiento global

In [10]:
# Saldo medio general
saldo_medio_global = df['balance'].mean()
print(round((saldo_medio_global/1000),1), "k€")

1.5 k€


In [11]:
# % de clientes con hipoteca
con_hipoteca = df['housing'].value_counts().get(True)
total = len(df['housing'])

hipoteca_pct = 100 * con_hipoteca / total
print(f"{hipoteca_pct:.2f}%")

47.63%


In [12]:
# % de clientes con préstamo
con_prestamo = df['loan'].value_counts().get(True)
total = len(df['loan'])

prestamo_pct = 100 * con_prestamo / total
print(f"{prestamo_pct:.2f}%")

13.10%


In [13]:
# % de clientes con ambos
con_ambos = len(df[df['loan'] & df['housing']])
total = len(df)

ambos_pct = 100 * con_ambos / total
print(f"{ambos_pct:.2f}%")

7.50%


In [14]:
# % contratacion deposito
contratan = df['deposit'].value_counts().get(True)
total = len(df['deposit'])

deposito_pct = 100 * contratan / total
print(f"{deposito_pct:.2f}%")

47.71%


In [15]:
# % de incumplimiento global
pd_global = df['default'].mean()
print("PD global =", (pd_global*100).round(2), "%")

PD global = 1.49 %


In [16]:
# Crear deciles y calcular métricas de riesgo
df['balance_decile'] = pd.qcut(df['balance'], q=10, labels=False)
default_rates = (
    df.groupby('balance_decile')['default']
    .agg(['mean', 'count'])
    .reset_index()
    .rename(columns={'mean': 'default', 'count': 'clientes'})
)
default_rates['balance_decile'] += 1
default_rates['default'] = default_rates['default'].round(4)
default_rates['indice_riesgo'] = (default_rates['default'] / pd_global).round(2)

# Calcular porcentaje de clientes por decil
clientes_totales = len(df)
default_rates['porcentaje_clientes'] = (default_rates['clientes'] / clientes_totales * 100).round(2)

# Clasificar nivel de riesgo
def categorize_risk(rate):
    return (
        'Muy alto' if rate > 0.10 else
        'Alto'     if rate > 0.05 else
        'Moderado' if rate > 0.02 else
        'Bajo'
    )

default_rates['risk_level'] = default_rates['default'].apply(categorize_risk)

# Porcentaje de clientes por categoría de riesgo
porcentaje_riesgo = (
    default_rates.groupby('risk_level')['porcentaje_clientes']
    .sum()
    .reindex(['Muy alto', 'Alto', 'Moderado', 'Bajo'])
    .reset_index()
)
porcentaje_riesgo['porcentaje_clientes'] = porcentaje_riesgo['porcentaje_clientes'].fillna(0).astype(int)
porcentaje_riesgo

,risk_level,porcentaje_clientes
0,Muy alto,0
1,Alto,13
2,Moderado,6
3,Bajo,79


## Regresión lineal para ver cómo afectan `housing` y `loan` al `balance`

In [17]:
# Copia de las columnas del df que voy a necesitar
df2 = df[['id','default', 'balance', 'housing', 'loan']].copy()
df2.head()

,id,default,balance,housing,loan
0,1,False,2343,True,False
1,2,False,2343,True,False
2,3,False,45,False,False
3,4,False,1270,True,False
4,5,False,2476,True,False


In [18]:
# Convierto los booleanos a variables binarias
df2[['default', 'housing', 'loan']].astype(int)

,default,housing,loan
0,0,1,0
1,0,1,0
2,0,0,0
3,0,1,0
4,0,1,0
...,...,...,...
16158,0,0,0
16159,0,1,0
16160,1,1,0
16161,0,0,0


In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [20]:
# Definir variables predictoras y objetivo
X = df2[['housing', 'loan']].astype(int)
y = df2['balance']

# Separar datos en entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelo
model = LinearRegression()
model.fit(X, y)

# Coeficientes
pd.DataFrame({
    'Variable': X.columns,
    'Coeficiente': model.coef_
})

,Variable,Coeficiente
0,housing,-420.579545
1,loan,-791.195527


Cada variable tiene un coeficiente que indica el impacto que tienen sobre la variable dependiente (`balance`).
- Housing: -420,6 €
    > Tener hipoteca se asocia con un saldo anual promedio de unos 420€ menos que no tenerla
- Loan: 791,2 €
    > Tener un préstamos personal se asocia con un saldo anual promedio 791€ menos

In [21]:
from sklearn.metrics import r2_score

y_pred = model.predict(X)
r2_manual = r2_score(y, y_pred)
print(f"R² manual: {r2_manual:.4f}")

R² manual: 0.0127


Del valor de R2 podemos obtener que este modelo solo explica el 1,27% de los datos, pero no explica la estadística

In [22]:
model = sm.OLS(y, X).fit()
print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:                balance   R-squared (uncentered):                   0.064
Model:                            OLS   Adj. R-squared (uncentered):              0.064
Method:                 Least Squares   F-statistic:                              552.3
Date:                Tue, 27 May 2025   Prob (F-statistic):                   9.71e-233
Time:                        09:33:43   Log-Likelihood:                     -1.5417e+05
No. Observations:               16163   AIC:                                  3.083e+05
Df Residuals:                   16161   BIC:                                  3.084e+05
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

Prob (F-statistic): si es < 0.05, significa que al menos una variable del modelo es significativa.